In [12]:
import requests, json, urllib.parse
base_url = 'https://itunes.apple.com/search'


artist = 'Агата Кристи'


params = {
    'term': artist,
    'limit': 200,    
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [13]:

import dateutil.parser
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

In [14]:
data = map(lambda x: {
    'artistName': x['artistName'],
    'kind' : x['kind'],
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionName'],
    'trackName': x['trackName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName']
}, data)
data = list(data)


In [15]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}:Hours,{minutes}:Minutes,{seconds}:Seconds"

def filter_list(items, value):
    has_more = items.count(value) > 1
    return [x for x in items if not (x == value and has_more)]


In [16]:
import statistics
countOfSongs = len(list(map(lambda x: x['trackName'], data)))
dataGenre = (list((map(lambda x: x['primaryGenreName'], data))))
dataAlbums = len(list(set(map(lambda x: x['collectionId'], data))))
songsDuration = sum(list(map(lambda x : x['trackTimeMillis'],data)))
Average_time = statistics.mean(list(map(lambda x : x['trackTimeMillis'],data)))
median_time = list(map(lambda x : x['trackTimeMillis'],data))
median_time.sort()
median_time = statistics.median(median_time)
songsCost = sum(filter_list(list(map(lambda x: x['trackPrice'], data)),-1))
Average_cost = statistics.mean(filter_list(list(map(lambda x: x['trackPrice'], data)),-1))
median_cost = filter_list(list(map(lambda x: x['trackPrice'], data)),-1)
median_cost.sort()
median_cost = statistics.median(median_cost)
time_extractor = lambda x: x['trackTimeMillis']
longest_song = max(data, key=time_extractor)
shortest_song = min(data, key=time_extractor)

track_price_extractor = lambda x: x['trackPrice']

dearest_song = max(data, key=track_price_extractor)

сheapest_song = min(data, key=track_price_extractor)

In [17]:

print('Artist:',artist)

print('Count of songs:',countOfSongs)

print('Most popular genre count:','Rock -',dataGenre.count('Rock'))

print('Count of albums:',dataAlbums)

print('Songs Duration:',convert_millis(songsDuration))

print('Average time:',convert_millis(Average_time))

print('Median duration songs:',convert_millis(median_time))

print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))

print('Total cost of songs ≈ ',round(songsCost),'$')

print('Average cost:',Average_cost,'$')

print('Median cost songs:',median_cost,'$')

print('The most expensive song:', dearest_song['trackName'],'Trackrpice:',dearest_song['trackPrice'])

if сheapest_song['trackPrice'] == -1 : 
    print('Сheapest song:', сheapest_song['trackName'],'Track price:','Free')
else:
    print('Сheapest song:', сheapest_song['trackName'],'Track price:',сheapest_song['trackPrice'])
    


Artist: Агата Кристи
Count of songs: 147
Most popular genre count: Rock - 147
Count of albums: 18
Songs Duration: 9:Hours,19:Minutes,54:Seconds
Average time: 0.0:Hours,3.0:Minutes,48.0:Seconds
Median duration songs: 0:Hours,3:Minutes,49:Seconds
Longest song: Халигаликришна 0:Hours,5:Minutes,41:Seconds
Shortest song: Собачье сердце 0:Hours,1:Minutes,48:Seconds
Total cost of songs ≈  189 $
Average cost: 1.2879591836734694 $
Median cost songs: 1.29 $
The most expensive song: Как на войне Trackrpice: 1.29
Сheapest song: Всё, как он сказал (Anton Sevidov Electro-Punk 1981 Remix) Track price: 0.99


In [18]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album

print(list(album_names.values()))
print(list(album_names.items()))

['Нечётный воин 2', 'Ностальгия', '10 лет жизни, Часть 1 (Live)', 'Отчёт 1983-1993', 'Избранное', 'Два kopablya', 'Скаzки', 'Heroin0', 'Декаданс', 'Коварство и любовь', 'Майн кайф?', 'Позорная звезда', 'Опиум', 'Триллер', 'Чудеса', 'Эпилог', 'Ураган', 'Нечётный воин. Лучшее']
[(292131249, 'Нечётный воин 2'), (1017226147, 'Ностальгия'), (1079904812, '10 лет жизни, Часть 1 (Live)'), (1080277953, 'Отчёт 1983-1993'), (1137064868, 'Избранное'), (1137066200, 'Два kopablya'), (1137072496, 'Скаzки'), (1137078503, 'Heroin0'), (1139743327, 'Декаданс'), (1139744137, 'Коварство и любовь'), (1139744676, 'Майн кайф?'), (1139745226, 'Позорная звезда'), (1139745875, 'Опиум'), (1140068164, 'Триллер'), (1140110420, 'Чудеса'), (1148877362, 'Эпилог'), (1420439709, 'Ураган'), (1518705709, 'Нечётный воин. Лучшее')]


In [22]:
from itertools import count
import dateutil.parser

album_name = album_names[1017226147]
album_values= list(album_groups[1017226147])


In [23]:

album_songs_count = len(list(set(map(lambda x: x['trackId'], album_values))))
dataGenre_Album = (list((map(lambda x: x['primaryGenreName'], album_values))))
songsDuration_Album = sum(list(map(lambda x : x['trackTimeMillis'],album_values)))
Average_time_Album = statistics.mean(list(map(lambda x : x['trackTimeMillis'],album_values)))
median_time_Album = list(map(lambda x : x['trackTimeMillis'],album_values))
median_time_Album.sort()
median_time_Album = statistics.median(median_time_Album)

longest_song_Album = max(album_values, key=time_extractor)
shortest_song_Album = min(album_values, key=time_extractor)
dearest_song_Album = max(album_values, key=track_price_extractor)
сheapest_song_Album = min(album_values, key=track_price_extractor)


songsCost_Album = sum(filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1))
Average_cost_Album = statistics.mean(filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1))
median_cost_Album = filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1)
median_cost_Album.sort()
median_cost_Album = statistics.median(median_cost_Album)

In [24]:
print('Album name:',album_name)

print('Count of songs:',album_songs_count)

print('Most popular genre count:','Rock -',dataGenre_Album.count('Rock'))

print('Songs Duration:',convert_millis(songsDuration_Album))

print('Average time:',convert_millis(Average_time_Album))

print('Median duration songs:',convert_millis(median_time_Album))

print('Longest song:', longest_song_Album['trackName'], convert_millis(longest_song_Album['trackTimeMillis']))

print('Shortest song:', shortest_song_Album['trackName'], convert_millis(shortest_song_Album['trackTimeMillis']))

print('Total cost of songs ≈ ',round(songsCost_Album),'$')

print('Average cost:',Average_cost_Album,'$')

print('Median cost songs:',median_cost_Album,'$')

print('The most expensive song:', dearest_song_Album['trackName'],'Trackrpice:',dearest_song_Album['trackPrice'])

if сheapest_song['trackPrice'] == -1 : 
    print('Сheapest song:', сheapest_song_Album['trackName'],'Track price:','Free')
else:
    print('Сheapest song:', сheapest_song_Album['trackName'],'Track price:',сheapest_song_Album['trackPrice'])

Album name: Ностальгия
Count of songs: 33
Most popular genre count: Rock - 33
Songs Duration: 2:Hours,7:Minutes,21:Seconds
Average time: 0.0:Hours,3.0:Minutes,51.0:Seconds
Median duration songs: 0:Hours,3:Minutes,56:Seconds
Longest song: Халигаликришна 0:Hours,5:Minutes,41:Seconds
Shortest song: В интересах революции 0:Hours,2:Minutes,39:Seconds
Total cost of songs ≈  43 $
Average cost: 1.29 $
Median cost songs: 1.29 $
The most expensive song: Никогда Trackrpice: 1.29
Сheapest song: Никогда Track price: 1.29
